In [ ]:
# !pwd
import os
import pandas as pd
import numpy as np

In [ ]:
all_sub_adjMat = np.load("corr_mat_abide_605.npy")
sub_names = np.load("filename_abide_605.npy", allow_pickle= True)
print(len(all_sub_adjMat), len(sub_names))

In [ ]:
with open('Phenotypic_V1_0b_preprocessed1.csv') as f:
    content = f.readlines()
content = [x.strip() for x in content]
content.pop(-1)
print(len(content))

In [ ]:
df = pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv') # can also index sheet by name or fetch all sheets
label = df['DX_GROUP'].tolist()
subject = df['FILE_ID'].tolist()
print(len(label),label[:5], subject[:5])

In [ ]:
subnames = [str(i)[:-14] for i in sub_names]
print(len(subnames), subnames[:5])

In [ ]:
adni90_feature = []
adni90_labels = []
adni90_adjmat = []
# edge_fea = []

for x,i in enumerate(subnames):
    ind = [n for n, l in enumerate(subject) if l.startswith(i)]
#     ind = subject.index(i)
    adni90_labels.append(label[ind[0]])
    adni90_feature.append(all_sub_adjMat[x])


for i in adni90_feature:
    temp = np.zeros((90, 90))
    for x in range(90):
        for y in range(90):
            if abs(i[x][y]) >= 0.15:
                temp[x][y] = 1
#                 edge_fea.append(i[x][y])
#             else:
#                 edge_fea.append(-999)
    adni90_adjmat.append(list(temp))

            
    
print(len(adni90_labels), len(adni90_adjmat), len(adni90_feature))
print(np.array(adni90_adjmat[0]).shape)

In [ ]:
avg = []
for k in range(605):
    count = 0
    for j in adni90_adjmat[k]:
#         print(len(j))
        for i in j:
            if i == 1:
                count += 1

    avg.append(count)
print(sum(avg)/len(avg))


In [ ]:
# # choose the classification type

lab1 = []
exp1 = []
fea1 = []

stop  = 0

for c,i in enumerate(adni90_labels):
    if i == 1:
        lab1.append(1)
        exp1.append(adni90_adjmat[c])
        
    else:
        lab1.append(2)
        exp1.append(adni90_adjmat[c])
        
print(len(exp1), len(fea1), len(lab1))

In [ ]:
# for ADNI_A.txt

edgeFea = []
count = -1
adni_a = []
for i, x in enumerate(exp1):
    temp = []
    for a in range(len(x)):
        for b in range(len(x[0])):
            count += 1
            if a < b and x[a][b] == 1:
                temp.append((a+1+i*90,b+1+i*90))
#                 edgeFea.append(edge_fea[count])

    adni_a.extend(temp)

print(len(adni_a))
print(adni_a[-10:])


with open("/home/anoopkumar/Brain/HGP-SL/data/ABIDE/raw/ABIDE_A.txt","w") as f:
    for item in adni_a:
        f.write("%s, %s\n" % (item[0],item[1]))

# print(len(edgeFea))

In [ ]:
print(len(exp1))

In [ ]:
# ADNI_graph_indicator.txt

node_graph_indicator = []
for i in range(1,len(exp1)+1):
    node_graph_indicator.extend([i for j in range(90)])

with open("/home/anoopkumar/Brain/HGP-SL/data/ABIDE/raw/ABIDE_graph_indicator.txt","w") as f:
    for item in node_graph_indicator:
        f.write("%s\n" % item) 

In [ ]:
# ADNI_graph_labels.txt

with open("/home/anoopkumar/Brain/HGP-SL/data/ABIDE/raw/ABIDE_graph_labels.txt","w") as f:
    for item in lab1:
        f.write("%s\n" % item) 

In [ ]:
# ADNI node labels.txt



node_labels = []

for i in range(len(exp1)):
    for j in range(90):
        node_labels.append(j)
print(len(node_labels), len(node_graph_indicator))                   
with open("/home/anoopkumar/Brain/HGP-SL/data/ABIDE/raw/ABIDE_node_labels.txt","w") as f:
    for item in node_labels:
        f.write("%s\n" % item) 

In [ ]:
import torch
import numpy as np
import pandas as pd
import os
import re
import sklearn

In [ ]:
print(torch.__version__, torch.cuda.is_available())

In [ ]:
!which pip

os.chdir("/home/anoopkumar/Brain/HGP-SL")
!pwd

In [ ]:
!rm -rf /home/anoopkumar/Brain/HGP-SL/data/ABIDE/processed
!python main.py --epochs 50 --dataset ABIDE --batch_size 64 --lr 0.0001 --dropout_ratio 0.8 --nhid 128
# --sparse_attention False --structure_learning False

In [ ]:
############### load learned weights and use it to train adni #######################

In [ ]:
!rm -rf /home/anoopkumar/Brain/HGP-SL/data/ADNI_612/processed
!python mainTL.py --epochs 20 --dataset ADNI_612 --batch_size 8 --lr 0.0001 --dropout_ratio 0.5 --nhid 256 --sparse_attention False --structure_learning False